In [1]:
import pandas as pd
import numpy as np
import time
import nba_api
np.set_printoptions(suppress=True)

from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif, chi2, RFE,f_regression, mutual_info_regression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
import importnb
#import data_cleaning_setup.database_interaction
#from data_cleaning_setup.database_interaction import connect_to_db

In [3]:
player_df = pd.read_parquet('data/players_stats_by_season.parquet')

FileNotFoundError: [Errno 2] No such file or directory: 'data/players_stats_by_season.parquet'

In [4]:
player_df

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,sp_work_PACE_RANK,PIE_RANK,FGM_RANK,FGA_RANK,FGM_PG_RANK,FGA_PG_RANK,FG_PCT_RANK,CFID,CFPARAMS,SEASON
0,203932,Aaron Gordon,Aaron,1610612743,DEN,26.0,75,46,29,0.613,...,463,218,55,69,81,106,106,5,"203932,1610612743",2021-22
1,1630565,Aaron Henry,Aaron,1610612755,PHI,22.0,6,6,0,1.000,...,47,596,561,557,573,581,569,5,"1630565,1610612755",2021-22
2,1628988,Aaron Holiday,Aaron,1610612756,PHX,25.0,63,34,29,0.540,...,400,272,257,250,320,310,287,5,"1628988,1610612756",2021-22
3,1630174,Aaron Nesmith,Aaron,1610612738,BOS,22.0,52,32,20,0.615,...,312,510,361,351,440,434,443,5,"1630174,1610612738",2021-22
4,1630598,Aaron Wiggins,Aaron,1610612760,OKC,23.0,50,13,37,0.260,...,208,461,254,251,236,240,223,5,"1630598,1610612760",2021-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12302,335,Willie Anderson,Willie,1610612748,MIA,30.0,28,22,6,0.786,...,389,200,351,356,348,370,169,5,"335,1610612748",1996-97
12303,416,Willie Burton,Willie,1610612737,ATL,29.0,24,15,9,0.625,...,303,394,329,320,286,241,403,5,"416,1610612737",1996-97
12304,1365,Xavier McDaniel,Xavier,1610612751,NJN,34.0,62,19,43,0.306,...,90,282,220,200,227,203,358,5,"1365,1610612751",1996-97
12305,212,Yinka Dare,Yinka,1610612751,NJN,24.0,41,12,29,0.293,...,62,419,373,367,426,420,396,5,"212,1610612751",1996-97


In [5]:
player_df.dtypes[30:]

USG_PCT                    float64
E_USG_PCT                  float64
E_PACE                     float64
PACE                       float64
PACE_PER40                 float64
sp_work_PACE               float64
PIE                        float64
POSS                         int64
FGM                          int64
FGA                          int64
FGM_PG                     float64
FGA_PG                     float64
FG_PCT                     float64
GP_RANK                      int64
W_RANK                       int64
L_RANK                       int64
W_PCT_RANK                   int64
MIN_RANK                     int64
E_OFF_RATING_RANK            int64
OFF_RATING_RANK              int64
sp_work_OFF_RATING_RANK      int64
E_DEF_RATING_RANK            int64
DEF_RATING_RANK              int64
sp_work_DEF_RATING_RANK      int64
E_NET_RATING_RANK            int64
NET_RATING_RANK              int64
sp_work_NET_RATING_RANK      int64
AST_PCT_RANK                 int64
AST_TO_RANK         

In [21]:
#Create dataset for modeling
player_model_df = player_df.drop(['GP_RANK',
'W_RANK'                       ,
'L_RANK'                       ,
'W_PCT_RANK'                   ,
'MIN_RANK'                     ,
'E_OFF_RATING_RANK'            ,
'OFF_RATING_RANK'              ,
'sp_work_OFF_RATING_RANK'      ,
'E_DEF_RATING_RANK'            ,
'DEF_RATING_RANK'              ,
'sp_work_DEF_RATING_RANK'      ,
'E_NET_RATING_RANK'            ,
'NET_RATING_RANK'              ,
'sp_work_NET_RATING_RANK'      ,
'AST_PCT_RANK'                 ,
'AST_TO_RANK'                  ,
'AST_RATIO_RANK'               ,
'OREB_PCT_RANK'                ,
'DREB_PCT_RANK'                ,
'REB_PCT_RANK'                 ,
'TM_TOV_PCT_RANK'              ,
'E_TOV_PCT_RANK'               ,
'EFG_PCT_RANK'                 ,
'TS_PCT_RANK'                  ,
'USG_PCT_RANK'                 ,
'E_USG_PCT_RANK'               ,
'E_PACE_RANK'                  ,
'PACE_RANK'                    ,
'sp_work_PACE_RANK'            ,
'PIE_RANK'                     ,
'FGM_RANK'                     ,
'FGA_RANK'                     ,
'FGM_PG_RANK'                  ,
'FGA_PG_RANK'                  ,
'FG_PCT_RANK'                  ,
'sp_work_OFF_RATING',
'sp_work_DEF_RATING',
'sp_work_NET_RATING',
'OFF_RATING',
'DEF_RATING',
'NET_RATING',
    
    
    'CFID', 'CFPARAMS', 'SEASON', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'NICKNAME', 'W', 'L'], axis = 1)

In [22]:
#Create Feature and Target DataSets
features = player_model_df.drop(['W_PCT'], axis = 1)
target = player_model_df['W_PCT']

#Splitting data into train/test
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.30, random_state=0)

#Scaling the data
scaler = MinMaxScaler()


X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns, index = X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns, index = X_test.index)

In [23]:
X_train_scaled

,AGE,GP,MIN,E_OFF_RATING,E_DEF_RATING,E_NET_RATING,AST_PCT,AST_TO,AST_RATIO,OREB_PCT,...,PACE,PACE_PER40,sp_work_PACE,PIE,POSS,FGM,FGA,FGM_PG,FGA_PG,FG_PCT
5965,0.346154,0.071429,0.059633,0.303333,0.4372,0.421091,0.167,0.095238,0.200,0.050,...,0.072309,0.072342,0.072309,0.330333,0.005003,0.000000,0.002301,0.000000,0.025180,0.000
8395,0.269231,0.059524,0.318807,0.298000,0.4552,0.410182,0.087,0.063333,0.138,0.056,...,0.064627,0.064647,0.064627,0.330333,0.021543,0.006135,0.010124,0.081967,0.133094,0.273
1666,0.346154,0.583333,0.240826,0.345333,0.4264,0.449091,0.196,0.117143,0.228,0.011,...,0.106042,0.106048,0.106042,0.348500,0.156220,0.080777,0.086516,0.131148,0.136691,0.420
8147,0.307692,0.416667,0.332569,0.344667,0.4304,0.447091,0.042,0.023810,0.069,0.079,...,0.070940,0.070939,0.070940,0.343667,0.138013,0.062372,0.068569,0.139344,0.147482,0.409
313,0.269231,0.595238,0.371560,0.342333,0.4468,0.438182,0.105,0.054762,0.125,0.025,...,0.101934,0.101960,0.101934,0.344333,0.240723,0.122699,0.138058,0.196721,0.212230,0.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4859,0.230769,0.928571,0.614679,0.344667,0.4300,0.447091,0.243,0.107619,0.225,0.014,...,0.089676,0.089696,0.089676,0.351000,0.592078,0.374233,0.382881,0.377049,0.377698,0.440
3264,0.461538,0.833333,0.763761,0.343667,0.4252,0.448545,0.097,0.083810,0.134,0.010,...,0.081059,0.081079,0.081059,0.346000,0.646282,0.361963,0.367694,0.409836,0.406475,0.443
9845,0.538462,0.595238,0.552752,0.336667,0.4108,0.451455,0.144,0.115714,0.220,0.006,...,0.067767,0.067813,0.067767,0.346167,0.322724,0.149284,0.150023,0.237705,0.230216,0.448
10799,0.384615,0.369048,0.337156,0.296333,0.4160,0.426909,0.065,0.022857,0.069,0.052,...,0.087739,0.087772,0.087739,0.345667,0.131897,0.068507,0.072250,0.172131,0.176259,0.427


In [30]:
#Find the 10 best features using chi2
selected_features_chi = SelectKBest(f_regression, k=9).fit(X_train_scaled, y_train)

#display the scoring for the features
print(pd.DataFrame(selected_features_chi.scores_, index = X_train_scaled.columns)[:50])

#Get a dataframe of selected top 10 features using chi2
cols_chi2 = selected_features_chi.get_support(indices=True)
X_train_kbest_chi2 = X_train_scaled.iloc[:,cols_chi2]
X_test_kbest_chi2 = X_test_scaled.iloc[:,cols_chi2]

                       0
AGE           200.858574
GP            179.968785
MIN             2.489419
E_OFF_RATING  297.694720
E_DEF_RATING   94.544820
E_NET_RATING  397.646222
AST_PCT         3.301176
AST_TO         29.166009
AST_RATIO       5.687433
OREB_PCT        3.850490
DREB_PCT        0.261538
REB_PCT        11.199610
TM_TOV_PCT      0.082391
E_TOV_PCT       0.094109
EFG_PCT       100.364319
TS_PCT         87.209375
USG_PCT         0.588839
E_USG_PCT       4.113160
E_PACE          7.537981
PACE            1.468300
PACE_PER40      1.466696
sp_work_PACE    1.468400
PIE             8.671427
POSS           94.241149
FGM           100.844280
FGA            69.674167
FGM_PG         23.404335
FGA_PG          3.559516
FG_PCT         60.470198


In [31]:
list(X_train_kbest_chi2.columns)

['AGE',
 'GP',
 'E_OFF_RATING',
 'E_DEF_RATING',
 'E_NET_RATING',
 'EFG_PCT',
 'TS_PCT',
 'POSS',
 'FGM']